In [1]:
!pipenv install pandas spacy

Installing pandas...
⠧ Installing pandas.....^C
⠙ Installing pandas...

Aborted!


In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 393.1 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import pandas as pd
import numpy as np
import spacy
import re

In [5]:
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words
stopwords.add("-")

In [6]:
def remove_references_nums(refs):
      cleaned_refs = []
      cib_ref_number_regex = r"^FT.{12}$"
      cib_ref_number_pattern = re.compile(cib_ref_number_regex)

      generic_ref_regex = r"\w*\d\w*[a-zA-Z]\w*|\w*[a-zA-Z]\w*\d\w*"
      generic_ref_pattern = re.compile(generic_ref_regex)

      numbers_regex = r"\d+"
      numbers_pattern = re.compile(numbers_regex)
      
      for ref in refs:
        if cib_ref_number_pattern.match(ref):
            continue
        elif generic_ref_pattern.match(ref):
            continue
        elif numbers_pattern.match(ref):
            continue
        else:
            cleaned_refs.append(ref)
      return cleaned_refs

In [7]:
def remove_stopwords(s):
    q = []
    for i in s:
        if i.lower() not in stopwords:
          q.append(i)
    return q

In [8]:
def ref_ipn_check(s):
    ipn_regex = r"^FT\w{10}\s-\sPayee Name:"
    cib_online_transfer = "Online Transfer-"

    if re.match(ipn_regex, s):
        return True

    if re.match(cib_online_transfer, s):
        return True

    return False

In [8]:
df = pd.read_csv("CIBStatement.csv")

In [9]:
s_data = df[['Narrative', 'Debit', 'Credit']]

In [10]:
s_data["Narrative_Tokenized"] = s_data["Narrative"].apply(lambda x: None if ref_ipn_check(x) else x.split(" "))

<ipython-input-10-31d4c1c9249c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data["Narrative_Tokenized"] = s_data["Narrative"].apply(lambda x: None if ref_ipn_check(x) else x.split(" "))


In [11]:
s_data.iloc[:]

,Narrative,Debit,Credit,Narrative_Tokenized
0,FT23005LF8V6 - COOK DOOR MENYA 18HEG 00320725 ...,138.00,NaN,"[FT23005LF8V6, -, COOK, DOOR, MENYA, 18HEG, 00..."
1,FT23005ZYXS1 - MOBIL CAIRO EG 5264390000005656,200.08,NaN,"[FT23005ZYXS1, -, MOBIL, CAIRO, EG, 5264390000..."
2,FT23005D13MY - Talabat Cairo EG 526805 5264390...,143.84,NaN,"[FT23005D13MY, -, Talabat, Cairo, EG, 526805, ..."
3,FT23009XWHYH - ACH(414OTDB230098010) RTIN(مصرو,NaN,"12,000.00","[FT23009XWHYH, -, ACH(414OTDB230098010), RTIN(..."
4,FT23009KYP3J - PAYMOB Abd Elrah CAIRO EG 52643...,106.00,NaN,"[FT23009KYP3J, -, PAYMOB, Abd, Elrah, CAIRO, E..."
...,...,...,...,...
87,FT23088LZT4R - BELGIUM FOR SWEETS THO CAIRO N ...,425.00,NaN,"[FT23088LZT4R, -, BELGIUM, FOR, SWEETS, THO, C..."
88,FT23088WZBVX - CHILLOUT SADAT 1 CAIRO N A 07BE...,205.00,NaN,"[FT23088WZBVX, -, CHILLOUT, SADAT, 1, CAIRO, N..."
89,100038175321 - Credit Interest,NaN,1.08,"[100038175321, -, Credit, Interest]"
90,FT230927K18J - Orange Egypt 1 EG 517460 526439...,115.00,NaN,"[FT230927K18J, -, Orange, Egypt, 1, EG, 517460..."


In [12]:
s_data.dropna(subset="Narrative_Tokenized", inplace=True)

<ipython-input-12-e40aee870fde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data.dropna(subset="Narrative_Tokenized", inplace=True)


In [13]:
s_data.iloc[:]

,Narrative,Debit,Credit,Narrative_Tokenized
0,FT23005LF8V6 - COOK DOOR MENYA 18HEG 00320725 ...,138.00,NaN,"[FT23005LF8V6, -, COOK, DOOR, MENYA, 18HEG, 00..."
1,FT23005ZYXS1 - MOBIL CAIRO EG 5264390000005656,200.08,NaN,"[FT23005ZYXS1, -, MOBIL, CAIRO, EG, 5264390000..."
2,FT23005D13MY - Talabat Cairo EG 526805 5264390...,143.84,NaN,"[FT23005D13MY, -, Talabat, Cairo, EG, 526805, ..."
3,FT23009XWHYH - ACH(414OTDB230098010) RTIN(مصرو,NaN,"12,000.00","[FT23009XWHYH, -, ACH(414OTDB230098010), RTIN(..."
4,FT23009KYP3J - PAYMOB Abd Elrah CAIRO EG 52643...,106.00,NaN,"[FT23009KYP3J, -, PAYMOB, Abd, Elrah, CAIRO, E..."
...,...,...,...,...
87,FT23088LZT4R - BELGIUM FOR SWEETS THO CAIRO N ...,425.00,NaN,"[FT23088LZT4R, -, BELGIUM, FOR, SWEETS, THO, C..."
88,FT23088WZBVX - CHILLOUT SADAT 1 CAIRO N A 07BE...,205.00,NaN,"[FT23088WZBVX, -, CHILLOUT, SADAT, 1, CAIRO, N..."
89,100038175321 - Credit Interest,NaN,1.08,"[100038175321, -, Credit, Interest]"
90,FT230927K18J - Orange Egypt 1 EG 517460 526439...,115.00,NaN,"[FT230927K18J, -, Orange, Egypt, 1, EG, 517460..."


In [14]:
s_data["Narrative_Tokenized"] = s_data["Narrative_Tokenized"].apply(lambda x: remove_stopwords(x))

<ipython-input-14-696d0d54d7d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data["Narrative_Tokenized"] = s_data["Narrative_Tokenized"].apply(lambda x: remove_stopwords(x))


In [15]:
s_data["Narrative_Tokenized"] = s_data["Narrative_Tokenized"].apply(lambda x: remove_references_nums(x))

<ipython-input-15-a10b8047da84>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data["Narrative_Tokenized"] = s_data["Narrative_Tokenized"].apply(lambda x: remove_references_nums(x))


In [16]:
s_data.iloc[:]

,Narrative,Debit,Credit,Narrative_Tokenized
0,FT23005LF8V6 - COOK DOOR MENYA 18HEG 00320725 ...,138.00,NaN,"[COOK, DOOR, MENYA]"
1,FT23005ZYXS1 - MOBIL CAIRO EG 5264390000005656,200.08,NaN,"[MOBIL, CAIRO, EG]"
2,FT23005D13MY - Talabat Cairo EG 526805 5264390...,143.84,NaN,"[Talabat, Cairo, EG]"
3,FT23009XWHYH - ACH(414OTDB230098010) RTIN(مصرو,NaN,"12,000.00","[ACH(414OTDB230098010), RTIN(مصرو]"
4,FT23009KYP3J - PAYMOB Abd Elrah CAIRO EG 52643...,106.00,NaN,"[PAYMOB, Abd, Elrah, CAIRO, EG]"
...,...,...,...,...
87,FT23088LZT4R - BELGIUM FOR SWEETS THO CAIRO N ...,425.00,NaN,"[BELGIUM, SWEETS, THO, CAIRO, N]"
88,FT23088WZBVX - CHILLOUT SADAT 1 CAIRO N A 07BE...,205.00,NaN,"[CHILLOUT, SADAT, CAIRO, N]"
89,100038175321 - Credit Interest,NaN,1.08,"[Credit, Interest]"
90,FT230927K18J - Orange Egypt 1 EG 517460 526439...,115.00,NaN,"[Orange, Egypt, EG]"


In [17]:
s_data["Clean_Narrative"] = s_data["Narrative_Tokenized"].apply(lambda x: " ". join(x))

<ipython-input-17-ccaa84db7fde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data["Clean_Narrative"] = s_data["Narrative_Tokenized"].apply(lambda x: " ". join(x))


In [18]:
s_data.iloc[20:50]

,Narrative,Debit,Credit,Narrative_Tokenized,Clean_Narrative
20,Outward - Online Transfer,"3,000.00",NaN,"[Outward, Online, Transfer]",Outward Online Transfer
21,FT23025LG64F - BRKA AIRO EG BRKN0026 526439000...,105.00,NaN,"[BRKA, AIRO, EG]",BRKA AIRO EG
22,FT23030ZC561 - Orange Egypt 1 EG 517460 526439...,143.00,NaN,"[Orange, Egypt, EG]",Orange Egypt EG
23,FT230302C0ZD - CHILLOUT AUC CAIRO N 07AEG 0031...,200.00,NaN,"[CHILLOUT, AUC, CAIRO, N]",CHILLOUT AUC CAIRO N
24,FT23031Z5FH8 - ACH(414OTDB230300006) RTIN(مصرو,NaN,"18,500.00","[ACH(414OTDB230300006), RTIN(مصرو]",ACH(414OTDB230300006) RTIN(مصرو
25,FT23031W33DJ - CIB MOBILE EL NARGS CAIRO EG 08...,"12,000.00",NaN,"[CIB, MOBILE, EL, NARGS, CAIRO, EG]",CIB MOBILE EL NARGS CAIRO EG
26,FT23031F5N9N - CIB MOBILE EL NARGS CAIRO EG 08...,"1,500.00",NaN,"[CIB, MOBILE, EL, NARGS, CAIRO, EG]",CIB MOBILE EL NARGS CAIRO EG
27,FT23031NSZ6T - MCDONALDS MOBIL ELNARG CAIRO N ...,192.50,NaN,"[MCDONALDS, MOBIL, ELNARG, CAIRO, N]",MCDONALDS MOBIL ELNARG CAIRO N
28,100038175321 - Credit Interest,NaN,0.82,"[Credit, Interest]",Credit Interest
29,FT23036ZSG24 - NBE ATM866 DAHAB EG NBE23530 52...,305.00,NaN,"[NBE, DAHAB, EG]",NBE DAHAB EG


In [19]:
clean_narrative = s_data

In [20]:
clean_narrative["Clean_Narrative"] = clean_narrative["Clean_Narrative"].apply(lambda x: np.nan if x == "" else x)

<ipython-input-20-6c82454417b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_narrative["Clean_Narrative"] = clean_narrative["Clean_Narrative"].apply(lambda x: np.nan if x == "" else x)


In [21]:
clean_narrative.dropna(subset="Clean_Narrative", inplace=True)

<ipython-input-21-800a27003b66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_narrative.dropna(subset="Clean_Narrative", inplace=True)


In [22]:
clean_narrative.tail(20)

,Narrative,Debit,Credit,Narrative_Tokenized,Clean_Narrative
67,FT23068SKRYQ - ACH(414OTDB230682001) RTIN(مصرو,NaN,"5,000.00","[ACH(414OTDB230682001), RTIN(مصرو]",ACH(414OTDB230682001) RTIN(مصرو
69,FT23068JX72M - EASY ACCOUNT MONTHLY SUBSCRIPTI...,10.00,NaN,"[EASY, ACCOUNT, MONTHLY, SUBSCRIPTION, F]",EASY ACCOUNT MONTHLY SUBSCRIPTION F
71,FT23071NMPFS - SUEZ ATM197 asser City EG ATM19...,505.00,NaN,"[SUEZ, asser, City, EG]",SUEZ asser City EG
72,FT2307115BVF - myfawry New Cairo EG NBE01611 5...,38.21,NaN,"[myfawry, New, Cairo, EG]",myfawry New Cairo EG
73,FT230740WF76 - myfawry New Cairo EG NBE01611 5...,109.64,NaN,"[myfawry, New, Cairo, EG]",myfawry New Cairo EG
74,FT230785DLND - E-Statement Quarterly fees for ...,7.50,NaN,"[E-Statement, Quarterly, fees]",E-Statement Quarterly fees
75,FT23078NQSXZ - FLAMINGO CAIRO N 07AEG 00153702...,57.50,NaN,"[FLAMINGO, CAIRO, N]",FLAMINGO CAIRO N
76,FT230799JLJ5 - MCDONALDS CHILL OUT AU CAIRO N ...,188.00,NaN,"[MCDONALDS, CHILL, AU, CAIRO, N]",MCDONALDS CHILL AU CAIRO N
77,FT23081BCDZP - MOBIL OIL CAIRO EG 526439000000...,208.00,NaN,"[MOBIL, OIL, CAIRO, EG]",MOBIL OIL CAIRO EG
78,FT230822Q1LP - BRKA AIRO EG BRKN0026 526439000...,255.00,NaN,"[BRKA, AIRO, EG]",BRKA AIRO EG


In [23]:
clean_narrative["Clean_Narrative"].to_csv('clean_narrative.txt', sep='\t', index=False)